In [1]:
import pandas as pd

In [2]:
file_path = "../dataset/click-through-rate-prediction/train.gz"
data_df = pd.read_csv(file_path, header=0, nrows=1000000).sample(frac=1)

In [3]:
train_count = int(len(data_df) * 0.9)

In [4]:
X_train = data_df.iloc[:train_count,2:]
y_train = data_df.iloc[:train_count,1]

In [5]:
X_val = data_df.iloc[train_count:,2:]
y_val = data_df.iloc[train_count:,1]

In [6]:
X_val.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
639726,14102103,1005,0,6399eda6,968765cd,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,19251,320,50,2201,3,35,100221,43
967064,14102105,1005,0,85f751fd,c4e18dd6,50e219e0,0885a933,2347f47a,0f2161f8,a99f214a,...,1,2,21667,320,50,2494,3,35,-1,112
780610,14102104,1005,0,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,18095,320,50,2060,3,39,100105,23
238536,14102101,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,15704,320,50,1722,0,35,-1,79
892775,14102104,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,15703,320,50,1722,0,35,-1,79


In [10]:
X_train.columns

Index(['hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip',
       'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [9]:
X_train.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]].to_list()

['hour',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20']

## Model Training

In [ ]:
!pip install catboost

In [8]:
# source https://www.kaggle.com/code/pointerfly/easy-catboost

from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.5,
    # task_type='GPU',
    loss_function='Logloss',
#     gpu_ram_part=0.9,
#     boosting_type='Plain',
#     max_ctr_complexity=2,
#     depth=6,
#     gpu_cat_features_storage='CpuPinnedMemory',
)

In [9]:
cat_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=cat_features,
    # verbose=10,
)

0:	learn: 0.4557959	test: 0.4560125	best: 0.4560125 (0)	total: 1.07s	remaining: 52.7s
1:	learn: 0.4028367	test: 0.4027672	best: 0.4027672 (1)	total: 1.74s	remaining: 41.7s
2:	learn: 0.3909379	test: 0.3907888	best: 0.3907888 (2)	total: 2.35s	remaining: 36.9s
3:	learn: 0.3812163	test: 0.3772588	best: 0.3772588 (3)	total: 2.9s	remaining: 33.4s
4:	learn: 0.3781520	test: 0.3725805	best: 0.3725805 (4)	total: 3.32s	remaining: 29.9s
5:	learn: 0.3767268	test: 0.3705149	best: 0.3705149 (5)	total: 3.85s	remaining: 28.2s
6:	learn: 0.3760746	test: 0.3695845	best: 0.3695845 (6)	total: 4.39s	remaining: 27s
7:	learn: 0.3752779	test: 0.3683552	best: 0.3683552 (7)	total: 4.88s	remaining: 25.6s
8:	learn: 0.3749753	test: 0.3679229	best: 0.3679229 (8)	total: 5.36s	remaining: 24.4s
9:	learn: 0.3746746	test: 0.3677107	best: 0.3677107 (9)	total: 5.89s	remaining: 23.6s
10:	learn: 0.3743297	test: 0.3673186	best: 0.3673186 (10)	total: 6.32s	remaining: 22.4s
11:	learn: 0.3738269	test: 0.3668235	best: 0.3668235 (1

In [12]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,app_id,22.757444
1,device_ip,17.983645
2,site_id,15.043044
3,site_domain,11.521526
4,C17,7.872775
5,C14,4.053024
6,device_model,3.969689
7,site_category,3.837459
8,device_id,2.782026
9,C21,1.972062


In [18]:
# Predict
preds = model.predict(
    X_val,
    prediction_type='Probability', 
    ntree_start=0, ntree_end=model.get_best_iteration(), 
    thread_count=-1
)


In [20]:
pd.DataFrame(preds)

,0,1
0,0.951567,0.048433
1,0.537359,0.462641
2,0.591169,0.408831
3,0.866793,0.133207
4,0.969202,0.030798
...,...,...
99995,0.811377,0.188623
99996,0.957576,0.042424
99997,0.989126,0.010874
99998,0.985147,0.014853


In [15]:
import pickle

with open('model/catboost-classifier.pickle', 'wb') as handle:
    pickle.dump(model, handle)